<a href="https://colab.research.google.com/github/yousenwang/langchain_llm/blob/main/gpt4all_locally.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.3/256.3 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.1/965.1 kB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
!pip show langchain

Name: langchain
Version: 0.0.215
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
# from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [4]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [5]:
loader = DirectoryLoader('./', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [6]:
len(documents)

11

In [7]:
documents[0]

Document(page_content=' \n1 / 5 \nThis document  belongs to Wareconn Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  1 wareconn standard operating procedure  \n \n \n \n \n \n \nwareconn standard operating procedure  \nRoot cause modification  SOP (WEB and API)  \n \nVersion : \nVersion  Date  Editor  Description  \nv 1.0  2023/05/15 Eric Sun First draft (English version)  \n \n \nContent  \nSOP ................................ ................................ ................................ ................................ . 2 \n1.1 Root cause modification (wareconn web)  ................................ .........  2 \n1.2 Root cause modification (API)  ................................ ...........................  3 \n1.3 Statistics report  ................................ ................................ ..................  4 \n \n \n \n \n', metadata={'source': '09.war

| file name | pages |
|---|---|
|09.wareconn-Modify root cause API.pdf|5|
|MSFT RMA request SOP v1.0.pdf|6|

# Split Docs into Chunks

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [8]:
len(texts)

13

In [10]:
texts[0]

Document(page_content='1 / 5 \nThis document  belongs to Wareconn Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  1 wareconn standard operating procedure  \n \n \n \n \n \n \nwareconn standard operating procedure  \nRoot cause modification  SOP (WEB and API)  \n \nVersion : \nVersion  Date  Editor  Description  \nv 1.0  2023/05/15 Eric Sun First draft (English version)  \n \n \nContent  \nSOP ................................ ................................ ................................ ................................ . 2 \n1.1 Root cause modification (wareconn web)  ................................ .........  2 \n1.2 Root cause modification (API)  ................................ ...........................  3 \n1.3 Statistics report  ................................ ................................ ..................  4', metadata={'source': '09.wareconn-Modify root 

# Embeddings

In [9]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [10]:

print(instructor_embeddings)

client=INSTRUCTOR(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: T5EncoderModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Dense({'in_features': 1024, 'out_features': 768, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (3): Normalize()
) model_name='hkunlp/instructor-xl' cache_folder=None model_kwargs={'device': 'cuda'} encode_kwargs={} embed_instruction='Represent the document for retrieval: ' query_instruction='Represent the question for retrieving supporting documents: '


# Local Knowledge DB

In [11]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [12]:
# persiste the db to disk
vectordb.persist()
vectordb = None


In [13]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

# Make a retriever

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
# docs = retriever.get_relevant_documents("What is Flash attention?")
docs = retriever.get_relevant_documents("What is the RMA request SOP?")

In [ ]:
len(docs)

In [ ]:
docs[0]

In [14]:
retriever = vectordb.as_retriever(search_kwargs={"k": 1})

In [15]:
retriever.search_type

'similarity'

In [16]:
retriever.search_kwargs

{'k': 1}

In [17]:
# docs = retriever.get_relevant_documents("What is Flash attention?")
docs = retriever.get_relevant_documents("What is the RMA request SOP?")

In [18]:
len(docs)

1

In [19]:
docs[0]

Document(page_content='2 / 6 \nThis document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure  \nSOP  \nI. This guide will walk you through the process of requesting  RMA  through the \nWareconn Customer Portal.  \nII. Details on how to fill in the Part Number (PN) and Serial Number (SN)  \n1.1 RMA request  procedure (Parts return)  \n1. System login  \n⚫ Module: https://www.wareconn.com/  \n⚫ Description:  Login  with account and password  \n \nTo log in, follow these steps:  \nGo to https://www.wareconn.com/ and click Log in  \n \nFill in your email and password, then click Login\n \nClick Enter  in the Customer portal', metadata={'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 1})

# LLM: GPT4All

In [5]:
!pip install wandb
import wandb
wandb.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=93073a6393f9f3688bfb83efbed34d37b5c8140171ee54c0eaf2d007fcf33af3
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [20]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [21]:
!pip install gpt4all

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.5 MB/s eta 0:00:00


In [22]:
llm_name = "ggml-gpt4all-j-v1.3-groovy.bin"

In [23]:
import gpt4all
gptj = gpt4all.GPT4All(llm_name)


100%|██████████| 3.79G/3.79G [02:36<00:00, 24.2MiB/s]


Model downloaded at:  /root/.cache/gpt4all/ggml-gpt4all-j-v1.3-groovy.bin


https://github.com/nomic-ai/gpt4all/blob/main/gpt4all-bindings/python/README.md

In [9]:
%%time
messages = [{"role": "user", "content": "Name 3 colors"}]
gptj.chat_completion(messages)

### Instruction: 
            The prompt below is a question to answer, a task to complete, or a conversation 
            to respond to; decide which and write an appropriate response.
            
### Prompt: 
Name 3 colors
### Response:
 
Blue, Green and Red
CPU times: user 1min 48s, sys: 195 ms, total: 1min 48s
Wall time: 1min 12s


{'model': 'ggml-gpt4all-j-v1.3-groovy',
 'usage': {'prompt_tokens': 239, 'completion_tokens': 21, 'total_tokens': 260},
 'choices': [{'message': {'role': 'assistant',
    'content': ' \nBlue, Green and Red'}}]}

In [11]:
# You can run raw generate as well on your input. But performance will degrade.
gptj.generate('Name three colors')

''

https://python.langchain.com/docs/modules/model_io/models/llms/integrations/gpt4all.html

In [24]:
local_path = (
    f"../root/.cache/gpt4all/{llm_name}"  # replace with your desired local file path
)

In [25]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]
# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)
# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
# llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

Found model file at  ../root/.cache/gpt4all/ggml-gpt4all-j-v1.3-groovy.bin
Found model file at  ../root/.cache/gpt4all/ggml-gpt4all-j-v1.3-groovy.bin


KeyboardInterrupt: ignored

In [25]:
# template = """Question: {question}

# Answer: Let's think step by step."""

template = """Question: {question}

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

# question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"
question = "Name 3 colors"

In [ ]:
! watch nvidia-smi

In [ ]:
%%time
llm_chain.run(question)

In [ ]:
%%time
llm_chain.run(question)

In [26]:

# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [27]:

## Cite sources

import textwrap
# @get_openai_callback()
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text
# @get_openai_callback()
def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
%%time
# full example
query = "What is the RMA request SOP?"
llm_response = qa_chain(query)


In [ ]:
print(llm_response)

In [ ]:
process_llm_response(llm_response)